In [2]:
# General imports
import os
import gzip
import numpy as np
import matplotlib.pyplot as plt
from pylab import cm
import warnings

warnings.filterwarnings("ignore")

# scikit-learn imports
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Qiskit imports
from qiskit import Aer, execute
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector
from qiskit.circuit.library import PauliFeatureMap, ZFeatureMap, ZZFeatureMap
from qiskit.circuit.library import TwoLocal, NLocal, RealAmplitudes, EfficientSU2
from qiskit.circuit.library import HGate, RXGate, RYGate, RZGate, CXGate, CRXGate, CRZGate
from qiskit_machine_learning.kernels import QuantumKernel

In [2]:
# Load MNIST dataset
#file = ".data/cifar-10-python.tar.gz"
#data = np.load(DATA_PATH)

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def cifar_10_reshape(batch_arg):
    output=np.reshape(batch_arg,(10000,32,32,3)).transpose(0,2,1,3)
    return output

batch1=unpickle('data1/images/data_batch_1')
batch2=unpickle('data1/images/data_batch_2')
batch3=unpickle('data1/images/data_batch_3')
batch4=unpickle('data1/images/data_batch_4')
batch5=unpickle('data1/images/data_batch_5')

#print(batch1)

batch1_data=cifar_10_reshape(batch1[b'data'])
batch2_data=cifar_10_reshape(batch2[b'data'])
batch3_data=cifar_10_reshape(batch3[b'data'])
batch4_data=cifar_10_reshape(batch4[b'data'])
batch5_data=cifar_10_reshape(batch5[b'data'])

batch1_labels=batch1[b'labels']
batch2_labels=batch2[b'labels']
batch3_labels=batch3[b'labels']
batch4_labels=batch4[b'labels']
batch5_labels=batch5[b'labels']

test_batch=unpickle('data1/images/test_batch')
test_images=cifar_10_reshape(test_batch[b'data'])
test_labels_data=test_batch[b'labels']

zsamples, zx, zy, zz = test_images.shape
sample_test = test_images.reshape((zsamples,zx*zy*zz))

train_images=np.concatenate((batch1_data,batch2_data,batch3_data,batch4_data,batch5_data),axis=0)
train_labels_data=np.concatenate((batch1_labels,batch2_labels,batch3_labels,batch4_labels,batch5_labels),axis=0)
train_images.shape
#train_labels_data.shape

(50000, 32, 32, 3)

In [3]:
#sample_train = data['data_batch_1']
#labels_train = data['data_batch_2']
#sample_test =  data['test_batch']
#n_features = 10
# Split train data
sample_train1, sample_val1, labels_train, labels_val = train_test_split(
    train_images, train_labels_data, test_size=0.2, random_state=42)

print(sample_train1.shape)
#print(sample_val.shape)
nsamples, nx, ny, nz = sample_train1.shape
#nsamples = 32
sample_train = sample_train1.reshape((nsamples,nx*ny*nz))

msamples, mx, my, mz = sample_val1.shape
sample_val = sample_val1.reshape((msamples,mx*my*mz))
print(nsamples)
#print(n_features)
# Visualize samples
fig = plt.figure()

LABELS = [1, 3, 8]
num_labels = len(LABELS)
for i in range(num_labels):
    ax = fig.add_subplot(1, num_labels, i+1)
    img = sample_train[labels_train==LABELS[i]][0].reshape((32,32,3))#(3072, 3072))
    ax.imshow(img, cmap="Greys")

Traceback (most recent call last):
  File "/tmp/ipykernel_131/226494499.py", line 7, in <module>
    train_images, train_labels_data, test_size=0.2, random_state=42)
NameError: name 'train_images' is not defined

Use %tb to get the full traceback.


In [1]:
# Standardize
standard_scaler = StandardScaler()
sample_train = standard_scaler.fit_transform(sample_train)
sample_val = standard_scaler.transform(sample_val)
sample_test = standard_scaler.transform(sample_test)

# Reduce dimensions
N_DIM = 5
pca = PCA(n_components=N_DIM)
sample_train = pca.fit_transform(sample_train)
sample_val = pca.transform(sample_val)
sample_test = pca.transform(sample_test)

# Normalize
min_max_scaler = MinMaxScaler((-1, 1))
sample_train = min_max_scaler.fit_transform(sample_train)
sample_val = min_max_scaler.transform(sample_val)
sample_test = min_max_scaler.transform(sample_test)

Traceback (most recent call last):
  File "/tmp/ipykernel_131/616559045.py", line 2, in <module>
    standard_scaler = StandardScaler()
NameError: name 'StandardScaler' is not defined

Use %tb to get the full traceback.


In [6]:
labels_train_0 = np.where(labels_train==0, 1, 0)
labels_val_0 = np.where(labels_val==0, 1, 0)
print(labels_val)
print(labels_val_0.shape)
print(f'Original validation labels:      {labels_val}')
print(f'Validation labels for 0 vs Rest: {labels_val_0}')

[7 8 0 ... 8 2 6]
(10000,)
Original validation labels:      [7 8 0 ... 8 2 6]
Validation labels for 0 vs Rest: [0 0 1 ... 0 0 0]


In [7]:
pauli_map_0 = ZZFeatureMap(feature_dimension=N_DIM, reps=3, entanglement='circular')
kernel_0 = QuantumKernel(feature_map=pauli_map_0, quantum_instance=Aer.get_backend('statevector_simulator'))
#pauli_map_0.draw

In [8]:
print(sample_train.shape)

(40000, 15)


In [9]:
zz_ckt = kernel_0.construct_circuit(sample_train[5],sample_train[6])
#zz_ckt.decompose().decompose().draw(output='mpl')
backend = Aer.get_backend('qasm_simulator')
job = execute(zz_ckt,backend,shots=8192,seed_simulator=1024,seed_transpiler=1024)
counts = job.result().get_counts(zz_ckt)
print(f"Transition amplitude: {counts['0'*N_DIM]/sum(counts.values())}")

Transition amplitude: 0.000244140625


In [10]:
svc_0 = SVC(kernel='precomputed', probability=True)
#sample_train = np.reshape(sample_train, (-1, 2))

In [ ]:
matrix_train_0 = kernel_0.evaluate(sample_train)

In [1]:
svc_0.fit(matrix_train_0, labels_train_0)

Traceback (most recent call last):
  File "/tmp/ipykernel_212/1663356809.py", line 1, in <module>
    svc_0.fit(matrix_train_0, labels_train_0)
NameError: name 'svc_0' is not defined

Use %tb to get the full traceback.


In [ ]:
pauli_map_0 = ZZFeatureMap(feature_dimension=N_DIM, reps=3, entanglement='circular')
kernel_0 = QuantumKernel(feature_map=pauli_map_0, quantum_instance=Aer.get_backend('statevector_simulator'))


svc_0 = SVC(kernel='precomputed', probability=True)

matrix_train_0 = kernel_0.evaluate(x_vec=sample_train)
svc_0.fit(matrix_train_0, labels_train_0)

matrix_val_0 = kernel_0.evaluate(x_vec=sample_val, y_vec=sample_train)
pauli_score_0 = svc_0.score(matrix_val_0, labels_val_0)
print(f'Accuracy of discriminating between label 0 and others: {pauli_score_0*100}%')

In [44]:
matrix_test_0 = kernel_0.evaluate(x_vec=sample_test, y_vec=sample_train)
pred_0 = svc_0.predict_proba(matrix_test_0)[:, 1]
print(f'Probability of label 0: {np.round(pred_0, 2)}')

Probability of label 0: [0.61 0.94 0.37 0.14 0.17 0.13 0.67 0.48 0.79 0.57 0.09 0.15 0.15 0.28
 0.24 0.08 0.12 0.73 0.23 0.15]


In [ ]:
labels_train_2 = np.where(labels_train==2, 1, 0)
labels_val_2 = np.where(labels_val==2, 1, 0)

print(f'Original validation labels:      {labels_val}')
print(f'Validation labels for 2 vs Rest: {labels_val_2}')



pauli_map_2 = ZZFeatureMap(feature_dimension=N_DIM, reps=3, entanglement='circular')
kernel_2 = QuantumKernel(feature_map=pauli_map_2, quantum_instance=Aer.get_backend('statevector_simulator'))


svc_2 = SVC(kernel='precomputed', probability=True)

matrix_train_2 = kernel_2.evaluate(x_vec=sample_train)
svc_2.fit(matrix_train_2, labels_train_2)

matrix_val_2 = kernel_2.evaluate(x_vec=sample_val, y_vec=sample_train)
pauli_score_2 = svc_2.score(matrix_val_2, labels_val_2)
print(f'Accuracy of discriminating between label 2 and others: {pauli_score_2*100}%')

Original validation labels:      [7 8 0 ... 8 2 6]
Validation labels for 2 vs Rest: [0 0 0 ... 0 1 0]


In [46]:
matrix_test_2 = kernel_2.evaluate(x_vec=sample_test, y_vec=sample_train)

pred_2 = svc_2.predict_proba(matrix_test_2)[:, 1]
print(f'Probability of label 2: {np.round(pred_2, 2)}')

Probability of label 2: [0.34 0.1  0.26 0.3  0.35 0.43 0.38 0.23 0.15 0.36 0.13 0.64 0.88 0.48
 0.09 0.81 0.33 0.32 0.48 0.71]


In [47]:
labels_train_3 = np.where(labels_train==3, 1, 0)
labels_val_3 = np.where(labels_val==3, 1, 0)

print(f'Original validation labels:      {labels_val}')
print(f'Validation labels for 3 vs Rest: {labels_val_3}')




pauli_map_3 = ZZFeatureMap(feature_dimension=N_DIM, reps=3, entanglement='circular')
kernel_3 = QuantumKernel(feature_map=pauli_map_3, quantum_instance=Aer.get_backend('statevector_simulator'))


svc_3 = SVC(kernel='precomputed', probability=True)

matrix_train_3 = kernel_3.evaluate(x_vec=sample_train)
svc_3.fit(matrix_train_3, labels_train_3)

matrix_val_3 = kernel_3.evaluate(x_vec=sample_val, y_vec=sample_train)
pauli_score_3 = svc_3.score(matrix_val_3, labels_val_3)
print(f'Accuracy of discriminating between label 3 and others: {pauli_score_3*100}%')

Original validation labels:      [3 3 2 0 3 0 3 2 3 2 2 3 2 2 2 3 0 2 3 3]
Validation labels for 3 vs Rest: [1 1 0 0 1 0 1 0 1 0 0 1 0 0 0 1 0 0 1 1]
Accuracy of discriminating between label 3 and others: 70.0%


In [48]:
matrix_test_3 = kernel_3.evaluate(x_vec=sample_test, y_vec=sample_train)

pred_3 = svc_3.predict_proba(matrix_test_3)[:, 1]
print(f'Probability of label 3: {np.round(pred_3, 2)}')

Probability of label 3: [0.16 0.18 0.31 0.55 0.64 0.63 0.09 0.15 0.24 0.12 0.95 0.18 0.08 0.31
 0.88 0.23 0.5  0.12 0.38 0.12]


In [49]:
pred_0_t = np.array([0.51, 0.85, 0.35, 0.18, 0.2 , 0.17, 0.56, 0.42, 0.66, 0.49, 0.13, 0.18, 0.18, 0.28, 0.26, 0.13, 0.16, 0.61, 0.25, 0.19])
pred_2_t = np.array([0.3 , 0.07, 0.21, 0.26, 0.31, 0.41, 0.36, 0.18, 0.11, 0.33, 0.08, 0.67, 0.92, 0.48, 0.06, 0.86, 0.3 , 0.28, 0.47, 0.75])
pred_3_t = np.array([0.11, 0.13, 0.26, 0.53, 0.63, 0.63, 0.06, 0.1 , 0.18, 0.08, 0.97, 0.13, 0.05, 0.25, 0.9 , 0.17, 0.47, 0.08, 0.34, 0.08])


pred_test = np.where(((pred_0_t > pred_2_t) & (pred_0_t > pred_3_t)),0,(np.where(((pred_2_t > pred_0_t) & (pred_2_t > pred_3_t)),2,3)))

print(f'Prediction: {pred_test}')

Prediction: [0 0 0 3 3 3 0 0 0 0 3 2 2 2 3 2 3 0 2 2]
